In [16]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import linear_model
import seaborn as sns
import random
import math

In [3]:
def analyze_r03(state, human, time, called_as_r04=False):
    if human:
        DATA_CSV = 'data/'+state+'_2a.csv'
    else:
        DATA_CSV = 'data/'+state+'_2b.csv'
    data_points = pd.read_csv(DATA_CSV)
    nr1 = data_points['carb']
    nr1 = nr1.iloc[1:len(nr1)]
    nr2 = data_points['py']
    nr2 = nr2.iloc[1:len(nr2)]
    data_points = data_points.iloc[0:len(data_points.index)-1]
    nr1.index = np.arange(len(nr1.index))
    nr2.index = np.arange(len(nr2.index))
    if time:
        data_points.loc[:,'growth'] = (nr1 - data_points['carb']) / (nr2 - data_points['py'])
    else:
        data_points.loc[:,'growth'] = nr1
    data_points.loc[:,'post_py'] = nr2
    if called_as_r04:
        data_points = data_points[data_points.post_py - data_points.py == 5] 
    else:
        data_points = data_points[data_points.post_py // 10000 == data_points.py // 10000]
    data_points.drop(['py','post_py'], axis=1, inplace=True)
    data_points.index = np.arange(len(data_points.index))
    data_points = data_points.loc[:,['carb','lat','lon','growth']]
    data_points = data_points.as_matrix().tolist()
    for i, elem in enumerate(data_points):
        elem[0] = [x for x in elem]
        elem[0].pop()
        elem[1] = elem[-1]
        data_points[i] = elem[0:2]
    N_GROUPS = 10
    random.shuffle(data_points)
    groups = []
    prev_cutoff = 0
    for i in np.arange(N_GROUPS):
        next_cutoff = (i + 1) * len(data_points) / N_GROUPS
        groups.append(data_points[prev_cutoff:next_cutoff])
        prev_cutoff = next_cutoff
    min_mse = float("inf")
    for i in np.arange(N_GROUPS):
        training = []
        test = []
        for j, group in enumerate(groups):
            if j==i:
                test = group
            else:
                training.extend(group)
        training = np.array(training).T.tolist()
        clf = linear_model.LinearRegression()
        clf.fit(training[0], training[1])
        coef = clf.coef_
        cons = clf.intercept_
        sq_error = 0.0
        for elem in test:
            predicted = sum(coef * elem[0]) + cons
            actual = elem[1]
            sq_error += (actual - predicted) ** 2
        mse = math.sqrt(sq_error/len(test))
        if mse < min_mse:
            min_mse = mse
    return min_mse

In [8]:
def analyze_r04(state, human, time):
    return analyze_r03(state, human, time, called_as_r04=True)

In [9]:
STATE = 'AL'
print analyze_r03(STATE, True, True)
print analyze_r04(STATE, True, True)

162128.004447
429.01509547


In [17]:
DATA_CSV = 'data/AL_2a.csv'
data_points = pd.read_csv(DATA_CSV)

In [18]:
nr1 = data_points['carb']
nr1 = nr1.iloc[1:len(nr1)]
nr2 = data_points['py']
nr2 = nr2.iloc[1:len(nr2)]
data_points = data_points.iloc[0:len(data_points.index)-1]
nr1.index = np.arange(len(nr1.index))
nr2.index = np.arange(len(nr2.index))
data_points.loc[:,'growth'] = (nr1 - data_points['carb']) / (nr2 - data_points['py'])
data_points.loc[:,'post_py'] = nr2

In [19]:
data_points = data_points[data_points.post_py - data_points.py == 5]

In [25]:
stats.mode((data_points['post_py'] - data_points['py']).tolist()).mode[0]

5